In [ ]:
pip install scikit-learn


In [ ]:
pip install nltk

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
netflix_df = pd.read_csv("netflixData.csv")
print(netflix_df.head())

In [ ]:
netflix_df.info()

In [ ]:
#Check if there are null values
print(netflix_df.isnull())

In [ ]:
#Check if there are null values
print(netflix_df.isnull().sum())

In [ ]:
required_nf_df = netflix_df[["Title", "Description", "Content Type", "Genres"  ]]
required_nf_df.head()

In [ ]:
required_nf_df.count()

In [ ]:
# Data cleaning, let's drop the rows containing the null values
required_nf_df.dropna()

In [ ]:
required_nf_df.count()

In [ ]:
#Cleaning the title column
import nltk
import re

In [ ]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))


In [ ]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [ ]:
required_nf_df["Title"] = required_nf_df["Title"].apply(clean)

In [ ]:
print(required_nf_df["Title"])

In [ ]:
print(required_nf_df.Title.sample(10))

In [ ]:
genreList = required_nf_df["Genres"].tolist()
#tfidf = text.TfidfVectorizer(input=genreList, stop_words="english")
tfidf = text.TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(genreList)
similarity = cosine_similarity(tfidf_matrix)

In [ ]:
indices = pd.Series(required_nf_df.index, index=required_nf_df['Title']).drop_duplicates()

In [ ]:
def netFlix_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    movieindices = [i[0] for i in similarity_scores]
    return required_nf_df['Title'].iloc[movieindices]

In [ ]:
print(netFlix_recommendation("fitoor"))